In [1]:
import yfinance as yf
import pandas as pd

# =========================
# FUNCTION: GAMMA SIGNAL
# =========================
def gamma_signal(stock_symbol):
    print("\n==============================")
    print(f"Analyzing: {stock_symbol}")
    print("==============================")

    ticker = yf.Ticker(stock_symbol)

    # -------------------------
    # PRICE DATA
    # -------------------------
    price_data = yf.download(stock_symbol, period="1mo")
    if isinstance(price_data.columns, pd.MultiIndex):
        price_data.columns = price_data.columns.get_level_values(0)

    current_price = price_data['Close'].iloc[-1]
    print("Current Price:", round(current_price, 2))

    # -------------------------
    # OPTION DATA
    # -------------------------
    expiry = ticker.options[0]
    option_chain = ticker.option_chain(expiry)
    calls = option_chain.calls
    calls['volume'] = calls['volume'].fillna(0)

    # -------------------------
    # ATM FILTER
    # -------------------------
    atm_calls = calls[
        (calls['strike'] >= current_price - 50) &
        (calls['strike'] <= current_price + 50)
    ].copy()

    # -------------------------
    # GAMMA SCORE
    # -------------------------
    atm_calls['gamma_score'] = atm_calls['volume'] * (atm_calls['openInterest'] + 1)

    # -------------------------
    # SIGNAL CLASSIFICATION
    # -------------------------
    def classify_signal(score):
        if score > 50000:
            return "HIGH"
        elif score > 20000:
            return "MEDIUM"
        else:
            return "LOW"

    atm_calls['gamma_signal'] = atm_calls['gamma_score'].apply(classify_signal)

    # -------------------------
    # TOP RESULTS
    # -------------------------
    top_signals = atm_calls.sort_values(
        by='gamma_score', ascending=False
    ).head(5)

    print("\n===== GAMMA SIGNALS =====")
    print(top_signals[['strike', 'gamma_score', 'gamma_signal']])

# =========================
# MULTIPLE STOCKS
# =========================
stocks = ["TSLA", "NVDA", "AAPL", "MSFT"]

for stock in stocks:
    gamma_signal(stock)



Analyzing: TSLA


[*********************100%***********************]  1 of 1 completed


Current Price: 437.5

===== GAMMA SIGNALS =====
    strike  gamma_score gamma_signal
59   430.0      76778.0         HIGH
63   440.0      44166.0       MEDIUM
65   445.0      35621.0       MEDIUM
67   450.0      32555.0       MEDIUM
61   435.0      24033.0       MEDIUM

Analyzing: NVDA


[*********************100%***********************]  1 of 1 completed


Current Price: 186.23

===== GAMMA SIGNALS =====
    strike  gamma_score gamma_signal
35   190.0     161030.0         HIGH
37   195.0     132652.0         HIGH
36   192.5      44421.0       MEDIUM
39   200.0      35388.0       MEDIUM
34   187.5      25698.0       MEDIUM

Analyzing: AAPL


[*********************100%***********************]  1 of 1 completed


Current Price: 255.53

===== GAMMA SIGNALS =====
    strike  gamma_score gamma_signal
27   260.0      44848.0       MEDIUM
26   257.5      26898.0       MEDIUM
29   265.0      23152.0       MEDIUM
28   262.5      14818.0          LOW
30   267.5       8283.0          LOW

Analyzing: MSFT


[*********************100%***********************]  1 of 1 completed


Current Price: 459.86

===== GAMMA SIGNALS =====
    strike  gamma_score gamma_signal
33   460.0       8329.0          LOW
35   465.0       8258.0          LOW
37   470.0       7730.0          LOW
34   462.5       5658.0          LOW
41   480.0       5608.0          LOW
